In [1]:
import gymnasium as gym


#定义环境
class MyWrapper(gym.Wrapper):

    def __init__(self):
        env = gym.make('CartPole-v1',render_mode="human")
        super().__init__(env)
        self.env = env

    def reset(self, seed=None):
        state, info = self.env.reset()
        return state, info

    def step(self, action):
        state, reward, done, truncated, info = self.env.step(action)
        return state, reward, done, truncated, info


MyWrapper().reset()

(array([ 0.00446594, -0.02108578, -0.03344151, -0.02559977], dtype=float32),
 {})

In [2]:
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import time


def test_multiple_env(dumm, N):
    if dumm:
        #DummyVecEnv,在单线程中运行多个环境, 速度更快
        env = DummyVecEnv([MyWrapper] * N)
    else:
        #SubprocVecEnv,在多线程中运行多个环境,  start_method='fork'
        env = SubprocVecEnv([MyWrapper] * N)

    start = time.time()

    #训练一个模型
    model = PPO('MlpPolicy', env, verbose=0).learn(total_timesteps=5000, progress_bar=True)

    print('消耗时间=', time.time() - start)

    #关闭环境
    env.close()

    #测试
    return evaluate_policy(model, MyWrapper(), n_eval_episodes=20)


In [3]:
test_multiple_env(dumm=True, N=2)

Output()

消耗时间= 90.45246577262878


/home/lianxin/.local/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(269.2, 163.44681092025013)

In [4]:
test_multiple_env(dumm=True, N=10)

Output()

消耗时间= 65.79507732391357


(334.75, 123.46087436917009)

In [5]:
test_multiple_env(dumm=False, N=2)

Output()

消耗时间= 88.97805142402649


(309.55, 186.10493679642138)

In [6]:
test_multiple_env(dumm=False, N=10)

Output()

消耗时间= 52.399301052093506


(345.9, 112.73903494353674)